In [1]:
import requests
import json
import pandas as pd
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt
from netAPI import pickleLoad, pickleDump
import warnings
warnings.filterwarnings('ignore')

/home/mingyi/anaconda3/envs/geo38/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [ ]:
#BART
stations = pd.read_csv('station_lat_lng.csv')
stations = gpd.GeoDataFrame(stations,geometry=gpd.points_from_xy(stations.lng, stations.lat), crs="EPSG:4326")
stations['geometry'] = stations.geometry.buffer(0.01)

In [7]:
def query_generator(bound):
    query = {'transportation':"""[out:json][timeout:30000];(
    node["amenity"~"bicycle_parking|bicycle_rental|boat_sharing|bus_station|car_rental|car_sharing|charging_station|ferry_terminal|fuel|motorcycle_parking|parking|parking_entrance|parking_space|taxi"](%s);
    way["amenity"~"bicycle_parking|bicycle_rental|boat_sharing|bus_station|car_rental|car_sharing|charging_station|ferry_terminal|fuel|motorcycle_parking|parking|parking_entrance|parking_space|taxi"](%s);
    relation["amenity"~"bicycle_parking|bicycle_rental|boat_sharing|bus_station|car_rental|car_sharing|charging_station|ferry_terminal|fuel|motorcycle_parking|parking|parking_entrance|parking_space|taxi"](%s);
    );
    out center;

    """%(bound,bound,bound),
    'restaurant':"""
    [out:json][timeout:30000];
    (
    node["amenity"~"bar|biergarten|cafe|fast_food|food_court|ice_cream|pub|restaurant"](%s);
    way["amenity"~"bar|biergarten|cafe|fast_food|food_court|ice_cream|pub|restaurant"](%s);
    relation["amenity"~"bar|biergarten|cafe|fast_food|food_court|ice_cream|pub|restaurant"](%s);
    );
    out center;

    """%(bound,bound,bound), 
    'health':
            """
    [out:json][timeout:30000];
    (
    node["amenity"~"baby_hatch|clinic|dentist|doctors|hospital|nursing_home|social_facility|veterinary"](%s);
    way["amenity"~"baby_hatch|clinic|dentist|doctors|hospital|nursing_home|social_facility|veterinary"](%s);
    relation["amenity"~"baby_hatch|clinic|dentist|doctors|hospital|nursing_home|social_facility|veterinary"](%s);
    );
    out center;

    """%(bound,bound,bound),

    'education':"""
    [out:json][timeout:30000];
    (
    node["amenity"~"college|driving_school|kindergarten|language_school|library|toy_library|research_institute|training|music_school|school|traffic_park|university"](%s);
    way["amenity"~"college|driving_school|kindergarten|language_school|library|toy_library|research_institute|training|music_school|school|traffic_park|university"](%s);
    relation["amenity"~"college|driving_school|kindergarten|language_school|library|toy_library|research_institute|training|music_school|school|traffic_park|university"](%s);
    );
    out center;

    """%(bound,bound,bound),
    'retail':"""
    [out:json][timeout:30000];
    (
    node[landuse=retail](%s);
    way[landuse=retail](%s);
    relation[landuse=retail](%s);
    );
    out center;

    """%(bound,bound,bound)
            }
    return query

In [8]:
def OSMdownload(ct_map,city):
    overpass_url = "http://overpass-api.de/api/interpreter"
    bound = ct_map.dissolve().bounds.values.tolist()[0]
    bound = [bound[1],bound[0],bound[3],bound[2]]
    bound = ','.join([str(i) for i in bound])
    print(bound)
    query = query_generator(bound)
    for amenity in query.keys():
        print(amenity)
        overpass_query = query[amenity]
        response = requests.get(overpass_url, 
                        params={'data': overpass_query})
        data = response.json()
        x = []
        y = []
#         name = []
        for element in data['elements']:

            if element['type'] == 'node':
                lon = element['lon']
                lat = element['lat']
                x.append(lon)
                y.append(lat)
#                 amenity.append(element['tags']['amenity'])
            elif 'center' in element:
                lon = element['center']['lon']
                lat = element['center']['lat']
                x.append(lon)
                y.append(lat)
#                 amenity.append(element['tags']['amenity'])

#         pd.DataFrame({'x':x,'y':y}).to_csv('LEHD/transportation.csv',index=False)
        df = pd.DataFrame({'x':x,'y':y})
        gdf = gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df.x, df.y), crs="EPSG:4326")
#         print(ct_map)
        ct_map = ct_map.merge(gdf.sjoin(ct_map,op='within',how='left').groupby(['GEOID']).count()['x'].\
        reset_index().rename(columns={'x':amenity}),on='GEOID',how='left')
        print(ct_map.shape)
    ct_map[['GEOID','transportation', 'restaurant',
       'health', 'education', 'retail']].fillna(0).to_csv('LEHD/%sPOI.csv'%city,index=False)
    return ct_map

In [178]:
path = 'LEHD/'

cities = [
    ('New York City', 'ny', ['New York County, NY', 'Queens County, NY','Kings County, NY','Bronx County, NY','Richmond County, NY']),
    ('Los Angeles', 'ca', ['Los Angeles County, CA']),
    ('Chicago', 'il', ['Cook County, IL']),
    ('Houston', 'tx', ['Harris County, TX']),
    ('Boston', 'ma', ['Suffolk County, MA', 'Middlesex County, MA']),
    ('Phoenix', 'az', ['Maricopa County, AZ']),
    ('Philadelphia', 'pa', ['Philadelphia County, PA']),
    ('San Antonio', 'tx', ['Bexar County, TX']),
    ('San Diego', 'ca', ['San Diego County, CA']),
    ('Dallas', 'tx', ['Dallas County, TX']),
    ('San Jose', 'ca', ['Santa Clara County, CA']),
    ('Austin', 'tx', ['Travis County, TX']),
]

for city,state, counties in cities:
    print(city,state,'processing LEHD')
    LEHD = pd.read_csv(f'LEHD/{city}_flow.csv')
#     LEHD.O = LEHD.O.astype(str)
#     LEHD.D = LEHD.D.astype(str)
    nodes = list(set(LEHD.O.unique().tolist()+LEHD.D.unique().tolist()))
    print(len(nodes))
    ct_map = gpd.read_file('LEHD/'+state+'.zip')
    ct_map.GEOID = ct_map.GEOID.astype(int)
    ct_map = ct_map.loc[ct_map.GEOID.isin(nodes)]
    ct_map = ct_map.to_crs('epsg:4326')
    poi = OSMdownload(ct_map,city)
#     break

New York City ny processing LEHD
2318
40.491167,-74.258754,40.917705,-73.700169
transportation
(2318, 14)
restaurant
(2318, 15)
health
(2318, 16)
education
(2318, 17)
retail
(2318, 18)
Los Angeles ca processing LEHD
2495
32.75004,-118.94518,34.823302,-117.646374
transportation
(2495, 14)
restaurant
(2495, 15)
health
(2495, 16)
education
(2495, 17)
retail
(2495, 18)
Chicago il processing LEHD
1331
41.469705,-88.26364099999999,42.154292,-87.52366099999999
transportation
(1331, 14)
restaurant
(1331, 15)
health
(1331, 16)
education
(1331, 17)
retail
(1331, 18)
Houston tx processing LEHD
1115
29.497339,-95.96073299999999,30.170606,-94.908492
transportation
(1115, 14)
restaurant
(1115, 15)
health
(1115, 16)
education
(1115, 17)
retail
(1115, 18)
Boston ma processing LEHD
591
42.156782,-71.89877299999999,42.736554,-70.920101
transportation
(591, 14)
restaurant
(591, 15)
health
(591, 16)
education
(591, 17)
retail
(591, 18)
Phoenix az processing LEHD
1009
32.505058999999996,-113.335075,34.0481

In [9]:
state = 'ny'
city = 'deep_gravity_data'
LEHD = pd.read_csv('deepgravity/new_york/processed/flows_oa.csv.zip')
nodes = list(set(LEHD.residence.unique().tolist()+LEHD.workplace.unique().tolist()))
print(len(nodes))
ct_map = gpd.read_file('LEHD/'+state+'.zip')
ct_map.GEOID = ct_map.GEOID.astype(int)
ct_map = ct_map.loc[ct_map.GEOID.isin(nodes)]
ct_map = ct_map.to_crs('epsg:4326')
poi = OSMdownload(ct_map,city)

4905
40.476577999999996,-79.761979,45.015865,-71.777491
transportation
(4401, 14)
restaurant
(4401, 15)
health
(4401, 16)
education
(4401, 17)
retail
(4401, 18)


In [179]:
base = 'https://lehd.ces.census.gov/data/lodes/LODES8/'
for city,state, counties in cities:
    fts = pd.read_csv('LEHD/%sPOI.csv'%city)
    print(city,fts.shape)
#     rac = pd.read_csv(base+ state + '/rac/'+state+'_rac_S000_JT00_2018.csv.gz')
#     rac['ct'] = rac.h_geocode.astype(str).apply(lambda x:x[:-4]).astype(int)
    wac = pd.read_csv(base+ state + '/wac/'+state+'_wac_S000_JT00_2018.csv.gz')
    wac['ct'] = wac.w_geocode.astype(str).apply(lambda x:x[:-4]).astype(int)
#     rac = rac.drop(columns=['h_geocode'])
    wac = wac.drop(columns=['w_geocode'])
#     rac = rac.groupby(['ct']).sum().reset_index()
    wac = wac.groupby(['ct']).sum().reset_index()
#     fts = fts.merge(rac,left_on='GEOID',right_on='ct',how='left')
#     fts = fts.drop(columns=['ct','createdate'])
    fts = fts.merge(wac,left_on='GEOID',right_on='ct',how='left')
    fts = fts.drop(columns=['ct','createdate'])
    print(fts.shape)
    fts = fts[fts.columns[:-10]]
    fts.fillna(0).to_csv('LEHD/%sfts.csv'%city,index=False)
#     break

New York City (2318, 6)
(2318, 57)
Los Angeles (2495, 6)
(2495, 57)
Chicago (1331, 6)
(1331, 57)
Houston (1115, 6)
(1115, 57)
Boston (591, 6)
(591, 57)
Phoenix (1009, 6)
(1009, 57)
Philadelphia (406, 6)
(406, 57)
San Antonio (375, 6)
(375, 57)
San Diego (736, 6)
(736, 57)
Dallas (645, 6)
(645, 57)
San Jose (408, 6)
(408, 57)
Austin (290, 6)
(290, 57)


In [3]:
base = 'https://lehd.ces.census.gov/data/lodes/LODES8/'
state = 'ny'
df = pd.read_csv(base+ state + '/wac/'+state+'_wac_S000_JT00_2018.csv.gz')

In [5]:
df.columns

Index(['w_geocode', 'C000', 'CA01', 'CA02', 'CA03', 'CE01', 'CE02', 'CE03',
       'CNS01', 'CNS02', 'CNS03', 'CNS04', 'CNS05', 'CNS06', 'CNS07', 'CNS08',
       'CNS09', 'CNS10', 'CNS11', 'CNS12', 'CNS13', 'CNS14', 'CNS15', 'CNS16',
       'CNS17', 'CNS18', 'CNS19', 'CNS20', 'CR01', 'CR02', 'CR03', 'CR04',
       'CR05', 'CR07', 'CT01', 'CT02', 'CD01', 'CD02', 'CD03', 'CD04', 'CS01',
       'CS02', 'CFA01', 'CFA02', 'CFA03', 'CFA04', 'CFA05', 'CFS01', 'CFS02',
       'CFS03', 'CFS04', 'CFS05', 'createdate'],
      dtype='object')

In [9]:
base = 'https://lehd.ces.census.gov/data/lodes/LODES8/'
city = 'New York City'
fts = pd.read_csv('LEHD/%sPOI.csv'%city)
print(city,fts.shape)
#     rac = pd.read_csv(base+ state + '/rac/'+state+'_rac_S000_JT00_2018.csv.gz')
#     rac['ct'] = rac.h_geocode.astype(str).apply(lambda x:x[:-4]).astype(int)
wac = pd.read_csv(base+ state + '/wac/'+state+'_wac_S000_JT00_2018.csv.gz')
wac['ct'] = wac.w_geocode.astype(str).apply(lambda x:x[:-4]).astype(int)
#     rac = rac.drop(columns=['h_geocode'])
wac = wac.drop(columns=['w_geocode'])
#     rac = rac.groupby(['ct']).sum().reset_index()
wac = wac.groupby(['ct']).sum().reset_index()
#     fts = fts.merge(rac,left_on='GEOID',right_on='ct',how='left')
#     fts = fts.drop(columns=['ct','createdate'])
fts = fts.merge(wac,left_on='GEOID',right_on='ct',how='left')
fts = fts.drop(columns=['ct','createdate'])
print(fts.shape)
fts = fts[fts.columns[:-10]]
#     break

New York City (2318, 6)
(2318, 57)


In [10]:
fts.colui

,GEOID,transportation,restaurant,health,education,retail,C000,CA01,CA02,CA03,...,CR05,CR07,CT01,CT02,CD01,CD02,CD03,CD04,CS01,CS02
0,36047000700,16.0,20.0,10.0,6.0,0.0,800.0,222.0,408.0,170.0,...,1.0,23.0,626.0,174.0,110.0,131.0,133.0,204.0,354.0,446.0
1,36047000900,44.0,31.0,9.0,8.0,0.0,13382.0,4025.0,6562.0,2795.0,...,30.0,335.0,9957.0,3425.0,1388.0,1964.0,2656.0,3349.0,5250.0,8132.0
2,36047001100,18.0,49.0,3.0,5.0,0.0,70705.0,6903.0,42693.0,21109.0,...,137.0,1561.0,57029.0,13676.0,8390.0,15969.0,20981.0,18462.0,48114.0,22591.0
3,36047001300,15.0,3.0,0.0,6.0,0.0,3067.0,754.0,1665.0,648.0,...,1.0,62.0,2454.0,613.0,231.0,459.0,687.0,936.0,1376.0,1691.0
4,36047002000,1.0,5.0,1.0,0.0,0.0,3606.0,579.0,2170.0,857.0,...,14.0,81.0,2327.0,1279.0,782.0,718.0,828.0,699.0,1398.0,2208.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2313,36081027900,7.0,6.0,0.0,1.0,0.0,627.0,191.0,320.0,116.0,...,4.0,12.0,327.0,300.0,106.0,97.0,114.0,119.0,329.0,298.0
2314,36081028000,0.0,0.0,0.0,0.0,0.0,430.0,37.0,182.0,211.0,...,0.0,3.0,390.0,40.0,71.0,82.0,125.0,115.0,238.0,192.0
2315,36081028100,3.0,11.0,0.0,0.0,0.0,587.0,136.0,320.0,131.0,...,1.0,6.0,319.0,268.0,107.0,104.0,119.0,121.0,298.0,289.0
2316,36081028200,0.0,2.0,0.0,0.0,0.0,121.0,9.0,76.0,36.0,...,1.0,0.0,96.0,25.0,23.0,22.0,34.0,33.0,92.0,29.0
